In [1]:
from pathlib import Path

import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
seed = 42
torch.manual_seed(seed)

CLASSES = [
    "N/A",
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "N/A",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "N/A",
    "backpack",
    "umbrella",
    "N/A",
    "N/A",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "N/A",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "couch",
    "potted plant",
    "bed",
    "N/A",
    "dining table",
    "N/A",
    "N/A",
    "toilet",
    "N/A",
    "tv",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "N/A",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush",
]


2022-11-23 19:26:31.149028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 19:26:31.683054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/matias/RECVIS:
2022-11-23 19:26:31.683082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 19:26:31.745781: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 19:26:33.922455: W

In [2]:
%reload_ext autoreload
%autoreload 2

In [4]:
class FullImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        self.files = [x for x in self.root_dir.glob("**/*") if x.is_file()]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx: int):
        img_path = self.files[idx]

        image = Image.open(img_path).convert("RGB")
        width = image.width
        height = image.height

        if self.transform is not None:
            image = self.transform(image)

        return {"image": image, "path": img_path, "width": width, "height": height}


transform = transforms.Compose(
    [
        transforms.Resize((800, 800)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)


def collate_batch(batch):
    images = torch.stack([elem["image"] for elem in batch])
    paths = [elem["path"] for elem in batch]
    widths = [elem["width"] for elem in batch]
    heights = [elem["height"] for elem in batch]
    return {"images": images, "paths": paths, "widths": widths, "heights": heights}


dataset = FullImageDataset("bird_dataset/train_images", transform=transform)
dataloader = DataLoader(
    dataset, batch_size=2, shuffle=False, num_workers=0, collate_fn=collate_batch
)


In [5]:
model = torch.hub.load("facebookresearch/detr", "detr_resnet50", pretrained=True)
model.eval()


Using cache found in /home/matias/.cache/torch/hub/facebookresearch_detr_main
/home/matias/.pyenv/versions/3.9.8/envs/MVA/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/matias/.pyenv/versions/3.9.8/envs/MVA/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DETR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_f

In [56]:
bird_index = CLASSES.index("bird")


with tqdm(dataloader, unit="batch") as vepoch:
    model.eval()
    for batch in vepoch:
        images = batch["images"]
        paths = batch["paths"]
        widths = batch["widths"]
        heights = batch["heights"]

        outputs = model(images)
        probas = outputs["pred_logits"].softmax(-1)[:, :, :-1]
        keep = probas.max(-1).values > 0.7
        for pred_boxes, pred_logits, path, width, height in zip(
            outputs["pred_boxes"], outputs["pred_logits"], paths, widths, heights
        ):
            probas = pred_logits.softmax(-1)[:, :-1]
            keep = probas[:, :].max(-1).values > 0.7
            class_index = probas[keep].argmax()
            im = Image.open(path).convert("RGB")

            parts = list(path.parts)
            parts[1] += "_cropped"
            cropped_path = Path(*parts)
            cropped_path.parents[0].mkdir(parents=True, exist_ok=True)

            if class_index.item() == bird_index:
                bboxes_scaled = rescale_bboxes(pred_boxes[keep], width, height)
                im = im.crop(bboxes_scaled[0].tolist())
            else:
                print(f"Image at {path} not cropped")
            im.save(cropped_path)


  0%|          | 0/541 [00:00<?, ?batch/s]/home/matias/.cache/torch/hub/facebookresearch_detr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
  0%|          | 0/541 [00:02<?, ?batch/s]


ValueError: 

In [59]:
def box_cxcywh_to_square_xyxy(x, width, height):
    x_c, y_c, w, h = x.unbind(1)
    h = max(h, w * width / height)
    w = max(w, h * height / width)
    print(w, h)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, width, height):
    b = box_cxcywh_to_square_xyxy(out_bbox, width, height)
    b = b * torch.tensor([width, height, width, height], dtype=torch.float32)
    return b


tensor([0.5055], grad_fn=<DivBackward0>) tensor([0.5343], grad_fn=<UnbindBackward0>)
bird_dataset/train_images/015.Lazuli_Bunting/Lazuli_Bunting_0045_14954.jpg
tensor([0.5156], grad_fn=<DivBackward0>) tensor([0.7734], grad_fn=<UnbindBackward0>)
bird_dataset/train_images/015.Lazuli_Bunting/Lazuli_Bunting_0032_14778.jpg
